# AI Agent API Test Notebook

This notebook tests two scenarios:
1. **1-shot question**: "What properties do we have in texas"
2. **Conversation flow**: "What are 5 properties that were lost in September 2025?" followed by "show me more"


In [7]:
import requests
import json
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
from pydomo import Domo
import duckdb
import warnings
warnings.filterwarnings('ignore')

env_path = Path.cwd() / ".env"
if env_path.exists():
    load_dotenv(env_path)


In [13]:
# Switch between local and production
BASE_URL = "http://localhost:8000"  # LOCAL - has the column fix
# BASE_URL = "https://app-ai-agent-v2.ambitiousdesert-4823611f.centralus.azurecontainerapps.io"  # PRODUCTION

USER_ID = "test_user"
API_KEY = os.getenv("AZURE_AGENT_API_KEY") or os.getenv("API_KEY")


In [14]:
pd.set_option('display.max_columns', None)
# Access your variables
DOMO_CLIENT_ID = os.getenv('DOMO_CLIENT_ID')
DOMO_SECRET_KEY = os.getenv('DOMO_SECRET_KEY')

# Initialize Domo client
domo = Domo(DOMO_CLIENT_ID, DOMO_SECRET_KEY, api_host='api.domo.com')

## Test Single Query

In [15]:
url = f"{BASE_URL}/api/v1/query"
question1 = [
    "What are properties that were lost in September 2025"
    # "Tell me about the Denver portfolio",
    # "What is the average occupancy in Denver" 
    # "Tell me about the kpis for continental tower" 
    # "Get me the average 90 day trailing occupancy for denver corporate operating office"
][0]

payload = {
    "query": question1,
    "user_id": USER_ID,
    # "agent_config": {
    #     "model": "google/gemini-2.5-pro"
    # }
}

headers = {"Content-Type": "application/json"}
if API_KEY:
    headers["X-API-Key"] = API_KEY

response1 = requests.post(url, json=payload, headers=headers, timeout=120)
response1.raise_for_status()
result1 = response1.json()


In [16]:
print("Question:", question1)
print("\nResponse:")
print(result1.get("final_response", "No response"))
if result1.get("sql_query"):
    print("\nSQL Query:")
    print(result1["sql_query"])


Question: What are properties that were lost in September 2025

Response:
There are 59 properties that were lost in September 2025. Here are 5 examples:

*   **Flats at West Village, The** (record_record_id_: c1285p002386)
*   **View on 10th** (record_record_id_: c1285p002340)
*   **Vista, The** (record_record_id_: c1020p002271)
*   **Post on Nord** (record_record_id_: c1020p002797)
*   **Avondale Farms Seniors** (record_record_id_: c1203p000034)

SQL Query:
SELECT * FROM "Operations | PDB Property Table | PROD" WHERE (record_pending_loss_date BETWEEN '2025-09-01' AND '2025-09-30')


In [17]:
result1

{'query_id': 'query_a2a6d07ed1f6458a',
 'conversation_id': 'conv_14a1676a11ab4d1a',
 'timestamp': '2025-11-25T15:11:56Z',
 'user_id': 'test_user',
 'steps': [{'name': 'search_columns', 'duration_ms': 0},
  {'name': 'generate_sql', 'duration_ms': 2087},
  {'name': 'execute_query', 'duration_ms': 1654}],
 'final_response': 'There are 59 properties that were lost in September 2025. Here are 5 examples:\n\n*   **Flats at West Village, The** (record_record_id_: c1285p002386)\n*   **View on 10th** (record_record_id_: c1285p002340)\n*   **Vista, The** (record_record_id_: c1020p002271)\n*   **Post on Nord** (record_record_id_: c1020p002797)\n*   **Avondale Farms Seniors** (record_record_id_: c1203p000034)',
 'sql_query': 'SELECT * FROM "Operations | PDB Property Table | PROD" WHERE (record_pending_loss_date BETWEEN \'2025-09-01\' AND \'2025-09-30\')',
 'data_sample': [['Maintenance Supervisor ID',
   'record_affordable_units',
   'record_latitude',
   'record_new_property_address__city',
   'S

Check in SQL

In [ ]:
print(result1['sql_query'])

In [ ]:
selected_dataset = result1['metadata']['selected_dataset']
selected_dataset = '123783d1-459b-41c8-87ba-6468c8f7edaf' # pdb
selected_dataset = '90339811-aa5c-4e35-835c-714f161ba93e' # kpi
print('selected_dataset', selected_dataset)
df = domo.ds_get(selected_dataset)
print(df.shape)
df.head(1) # Display first few rows of the dataframe

In [ ]:
# Replace the table name with your registered view
# modified_query = result1['sql_query'].replace("`Operations | PDB Property Table | PROD`", "df_view")
modified_query = result1['sql_query'].replace("Operations | Master Property Table KPIs | PROD", "df_view")

# Create connection
conn = duckdb.connect()
conn.register('df_view', df)

modified_query = """
SELECT *
FROM
    df_view
WHERE
    lower("record_property_name") LIKE '%continental tower%';
"""
modified_query = """
SELECT record_corporate_operating_office, SUM(units) AS units_churn 
FROM df_view 
WHERE (record_termination_date BETWEEN DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month') 
    AND (DATE_TRUNC('month', CURRENT_DATE) - INTERVAL '1 day')) 
    -- AND meas_mo = (SELECT MAX(meas_mo) FROM df_view)
GROUP BY record_corporate_operating_office 
ORDER BY units_churn DESC LIMIT 1
"""

modified_query = """
SELECT COUNT(DISTINCT record_look_up_code) 
FROM df_view
WHERE (record_termination_date >= '2024-10-01') 
    AND (record_termination_date < '2024-11-01')
"""

# Execute your dynamic query
result = conn.execute(modified_query).df()
conn.close()

result

In [ ]:
result[result.columns[0]].tolist()

### Initial Question


In [ ]:
question2 = "What are 5 properties that were lost in September 2025?"

payload2 = {
    "query": question2,
    "user_id": USER_ID
}

response2 = requests.post(url, json=payload2, headers=headers, timeout=120)
response2.raise_for_status()
result2 = response2.json()
conversation_id = result2.get("conversation_id")

In [ ]:
print("Question:", question2)
print("\nResponse:")
print(result2.get("final_response", "No response"))
if result2.get("sql_query"):
    print("\nSQL Query:")
    print(result2["sql_query"])
print("\nConversation ID:", conversation_id)
result2

In [ ]:
question3 = "show me more"

payload3 = {
    "query": question3,
    "user_id": USER_ID,
    "conversation_id": conversation_id
}

response3 = requests.post(url, json=payload3, headers=headers, timeout=120)
response3.raise_for_status()
result3 = response3.json()

In [ ]:
print("Follow-up Question:", question3)
print("\nResponse:")
print(result3.get("final_response", "No response"))
if result3.get("sql_query"):
    print("\nSQL Query:")
    print(result3["sql_query"])
print("\nConversation ID:", result3.get("conversation_id"))
result3